In [26]:
import pandas as pd
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [27]:
# 读取 Excel 文件中的 "Sheet1"
try:
    full_data = pd.read_excel('/home/aistudio/work/附件2(处理后).xlsx', sheet_name='Sheet1')
except Exception as e:
    print(f"Error reading Excel file: {e}")

# 将 "销售日期" 列转换为 datetime 格式，以便进行筛选
full_data['销售日期'] = pd.to_datetime(full_data['销售日期'])

# 筛选出 2023 年 6 月 24-30 日的数据
date_range_start = '2023-06-24'
date_range_end = '2023-06-30'
filtered_data = full_data[(full_data['销售日期'] >= date_range_start) & (full_data['销售日期'] <= date_range_end)]

# 按日期和单品名称分组，然后对销量进行求和
grouped_data = filtered_data.groupby(['销售日期', '单品名称'])['销量(千克)'].sum().reset_index()

# 使用 pivot 函数整理数据格式
pivot_data = grouped_data.pivot(index='销售日期', columns='单品名称', values='销量(千克)').fillna(0)

# 显示整理后的数据
print(pivot_data)

单品名称        七彩椒(2)    上海青  云南油麦菜  云南油麦菜(份)   云南生菜  云南生菜(份)  净藕(1)  双孢菇(盒)  \
销售日期                                                                        
2023-06-24   0.951  5.127  0.000      26.0  0.000     58.0  7.656    12.0   
2023-06-25   1.048  1.383  4.232      19.0  0.557     37.0  5.251     8.0   
2023-06-26   1.506  1.420  0.573      14.0  7.971      4.0  4.493    11.0   
2023-06-27   2.101  1.823  0.293      18.0  0.251     23.0  4.354    10.0   
2023-06-28   1.528  5.553  0.000      21.0  0.000     33.0  6.587    13.0   
2023-06-29   0.168  4.687  0.000      26.0  2.904     35.0  7.403     7.0   
2023-06-30   0.000  7.036  0.000      25.0  0.000     36.0  6.440     9.0   

单品名称        圆茄子(2)   外地茼蒿  ...  西峡花菇(1)   野生粉藕  金针菇(盒)    长线茄  青红杭椒组合装(份)  \
销售日期                       ...                                              
2023-06-24   0.867  3.680  ...    6.180  0.000    18.0  6.831         1.0   
2023-06-25   0.616  0.000  ...    4.271  0.000    11.0  2.493         2.0  

In [28]:
# pivot_data.to_excel('/home/aistudio/work/问题3一周订单.xlsx',index = False)

In [29]:
# Check if the data was successfully loaded or not
if full_data is not None:
    # Define the list of product names to keep
    product_names_to_keep = [
        "七彩椒(2)", "上海青", "云南油麦菜", "云南油麦菜(份)", "云南生菜", "云南生菜(份)", "净藕(1)", "双孢菇(盒)",
        "圆茄子(2)", "外地茼蒿", "奶白菜", "姜蒜小米椒组合装(小份)", "娃娃菜", "小皱皮(份)", "小米椒(份)", "小青菜(1)",
        "木耳菜", "木耳菜(份)", "枝江青梗散花", "洪湖藕带", "海鲜菇(包)", "白玉菇(袋)", "竹叶菜", "紫茄子(1)", 
        "紫茄子(2)", "红椒(2)", "红莲藕带", "红薯尖", "芜湖青椒(1)", "苋菜", "菜心", "菠菜", "菠菜(份)", 
        "菱角", "虫草花(份)", "螺丝椒", "螺丝椒(份)", "蟹味菇与白玉菇双拼(盒)", "西兰花", "西峡花菇(1)", 
        "野生粉藕", "金针菇(盒)", "长线茄", "青红杭椒组合装(份)", "青线椒(份)", "青茄子(1)", "高瓜(1)", "高瓜(2)", 
        "鲜木耳(份)"
    ]

    # Filter the data to only keep rows where the product name is in the list of product names to keep
    filtered_data_by_selected_products = full_data[full_data['单品名称'].isin(product_names_to_keep)]

    # Save this filtered data to a new Excel file
    output_file_path_selected_products = '/home/aistudio/work/问题3的数据集.xlsx'
    try:
        filtered_data_by_selected_products.to_excel(output_file_path_selected_products, index=False)
        save_status_selected_products = "Successfully saved the filtered data."
    except Exception as e:
        save_status_selected_products = f"An error occurred while saving the file: {str(e)}"
else:
    save_status_selected_products = "Failed to reload the data."

save_status_selected_products, output_file_path_selected_products if full_data is not None else None


('Successfully saved the filtered data.', '/home/aistudio/work/问题3的数据集.xlsx')

In [30]:
question3_df = filtered_data_by_selected_products
attachment4_df = pd.read_excel('/home/aistudio/data/data237885/附件4.xlsx', sheet_name='Sheet1')

# Step 1: Group data by '销售日期' and '单品名称', and calculate the statistics
grouped_df = question3_df.groupby(['销售日期', '单品名称', '单品编码']).agg({
    '销量(千克)': 'sum',
    '分类名称': 'first',
    '星期': 'first',
    '是否打折销售': 'sum'
}).reset_index()

# Rename the columns
grouped_df.rename(columns={
    '销量(千克)': '销量(千克) 总计',
    '是否打折销售': '打折次数'
}, inplace=True)

# Step 2: Merge the grouped data with attachment 4 data to add '损耗率'
merged_df = pd.merge(grouped_df, attachment4_df[['单品编码', '损耗率(%)']], on='单品编码', how='left')

# Calculate the number of sales orders for each item
sales_order_count = question3_df.groupby(['销售日期', '单品名称']).size().reset_index(name='销售单数')

# Combine all the data
final_df = pd.merge(merged_df, sales_order_count, on=['销售日期', '单品名称'], how='left')

# Calculate the average sale price for each item on each sale date
average_price_df = question3_df.groupby(['销售日期', '单品名称'])['销售单价(元/千克)'].mean().reset_index()
average_price_df.rename(columns={'销售单价(元/千克)': '平均销售单价(元/千克)'}, inplace=True)

# Merge this with the final dataframe
final_df_with_avg_price = pd.merge(final_df, average_price_df, on=['销售日期', '单品名称'], how='left')

# Save the final dataframe to a new Excel file
output_filepath = '/home/aistudio/work/问题3的训练集.xlsx'
final_df_with_avg_price.to_excel(output_filepath, index=False)


In [31]:
from prophet import Prophet
import pandas as pd

df = final_df_with_avg_price

final_predictions = pd.DataFrame(columns=['单品名称', '预测销量', '预测销量下限', '预测销量上限', '预测价格', '预测价格下限', '预测价格上限'])
final_predictions = final_predictions.reset_index(drop=True)
print(type(final_predictions))

<class 'pandas.core.frame.DataFrame'>


In [32]:
# Create holiday dataframe
holidays = pd.DataFrame({
  'holiday': ['chinese_new_year', 'chinese_new_year', 'chinese_new_year', 'chinese_new_year',
              'national_day', 'national_day', 'national_day', 'national_day',
              'mid_autumn_festival', 'mid_autumn_festival', 'mid_autumn_festival'],
  'ds': pd.to_datetime(['2020-01-25', '2021-02-12', '2022-02-01', '2023-01-22',
                        '2020-10-01', '2021-10-01', '2022-10-01', '2023-10-01',
                        '2020-10-01', '2021-09-21', '2022-09-10']),
  'lower_window': [-4, -4, -4, -4, -1, -1, -1, -1, -1, -1, -1],
  'upper_window': [0, 0, 0, 0, 6, 6, 6, 6, 0, 0, 0],
})

unique_items = df['单品名称'].unique()

for item in unique_items:
    # Filter the DataFrame based on the item
    df_filtered = df[df['单品名称'] == item]
    
    # Sales prediction
    df_sales = df_filtered[['销售日期', '销量(千克) 总计']].rename(columns={'销售日期': 'ds', '销量(千克) 总计': 'y'})
    prophet_sales = Prophet(holidays=holidays)
    prophet_sales.fit(df_sales)
    future_sales = prophet_sales.make_future_dataframe(periods=180)  # Extend to include 2023-07-01
    forecast_sales = prophet_sales.predict(future_sales)
    
    # Price prediction
    df_price = df_filtered[['销售日期', '平均销售单价(元/千克)']].rename(columns={'销售日期': 'ds', '平均销售单价(元/千克)': 'y'})
    prophet_price = Prophet(holidays=holidays)
    prophet_price.fit(df_price)
    future_price = prophet_price.make_future_dataframe(periods=180)  # Extend to include 2023-07-01
    forecast_price = prophet_price.predict(future_price)
    
    # Extract predictions for 2023-07-01
    predicted_sales = forecast_sales.loc[forecast_sales['ds'] == '2023-07-01', ['yhat', 'yhat_lower', 'yhat_upper']].values[0]
    predicted_price = forecast_price.loc[forecast_price['ds'] == '2023-07-01', ['yhat', 'yhat_lower', 'yhat_upper']].values[0]
    
    # Create a new row and append it to the final DataFrame
    new_row = pd.DataFrame([{
        '单品名称': item,
        '预测销量': predicted_sales[0],
        '预测销量下限': predicted_sales[1],
        '预测销量上限': predicted_sales[2],
        '预测价格': predicted_price[0],
        '预测价格下限': predicted_price[1],
        '预测价格上限': predicted_price[2]
    }])
    final_predictions = pd.concat([final_predictions, new_row]).reset_index(drop=True)

# Print the final DataFrame with all predictions
print(final_predictions)

16:21:41 - cmdstanpy - INFO - Chain [1] start processing
16:21:42 - cmdstanpy - INFO - Chain [1] done processing
16:21:42 - cmdstanpy - INFO - Chain [1] start processing
16:21:42 - cmdstanpy - INFO - Chain [1] done processing
16:21:43 - cmdstanpy - INFO - Chain [1] start processing
16:21:43 - cmdstanpy - INFO - Chain [1] done processing
16:21:43 - cmdstanpy - INFO - Chain [1] start processing
16:21:43 - cmdstanpy - INFO - Chain [1] done processing
16:21:44 - cmdstanpy - INFO - Chain [1] start processing
16:21:44 - cmdstanpy - INFO - Chain [1] done processing
16:21:44 - cmdstanpy - INFO - Chain [1] start processing
16:21:45 - cmdstanpy - INFO - Chain [1] done processing
16:21:45 - cmdstanpy - INFO - Chain [1] start processing
16:21:45 - cmdstanpy - INFO - Chain [1] done processing
16:21:46 - cmdstanpy - INFO - Chain [1] start processing
16:21:46 - cmdstanpy - INFO - Chain [1] done processing
16:21:46 - cmdstanpy - INFO - Chain [1] start processing
16:21:46 - cmdstanpy - INFO - Chain [1]

            单品名称       预测销量     预测销量下限     预测销量上限       预测价格     预测价格下限  \
0            上海青   6.961620   0.997894  13.115961   7.498187   5.581716   
1          云南油麦菜   7.490722  -3.183824  17.938253   6.478207   4.665511   
2           云南生菜  15.525545   4.256583  26.148474   7.831655   5.868148   
3           外地茼蒿   4.078180   2.310948   5.866280  12.036806  10.228298   
4            娃娃菜  15.569064  -5.288482  36.951721   6.637924   5.773273   
5            木耳菜   5.558811   2.976029   8.156573   6.438581   4.059504   
6           洪湖藕带   7.191630   5.615309   8.946027  18.353342  14.120586   
7            竹叶菜  15.961313   9.301315  22.630353   4.525375   2.500421   
8         紫茄子(2)  18.790064  10.258589  26.324571   6.915868   5.096365   
9            红薯尖   9.819662   3.149795  16.232927   6.666682   5.334164   
10            苋菜   6.509690   1.758768  11.328808   4.408854   2.470077   
11            菜心   6.634768   1.386778  12.052976   7.028382   5.178845   
12            菠菜   5.8379

In [33]:
final_predictions.to_excel('/home/aistudio/work/问题3预测.xlsx')

In [35]:
df_prediction = final_predictions
df_training = final_df_with_avg_price

# 读取“利润率预测_更新”文件
file_path_profit_margin = '/home/aistudio/work/利润率预测_更新.xlsx'
df_profit_margin = pd.read_excel(file_path_profit_margin)

# 读取“销量预测_更新”文件
file_path_sales_prediction = '/home/aistudio/work/销量预测_更新.xlsx'
df_sales_prediction = pd.read_excel(file_path_sales_prediction)

# 去除训练集中重复的单品名称和对应的损耗率
df_training_unique = df_training[['单品名称', '损耗率(%)', '分类名称']].drop_duplicates()

# 合并单品销量预测和损耗率
merged_data = pd.merge(df_prediction, df_training_unique, on='单品名称', how='left')

# 添加类别利润率到数据集中
# 只选择2023-07-01的利润率
profit_rate_20230701 = df_profit_margin[df_profit_margin['日期'] == '2023-07-01'].iloc[0, 1:]
profit_rate_20230701 = profit_rate_20230701.reset_index().rename(columns={'index': '分类名称', 0: '预测利润率'})

# 将预测利润率添加到合并的数据集中
merged_data = pd.merge(merged_data, profit_rate_20230701, on='分类名称', how='left')

# 保存整合后的数据
merged_data.to_excel('/home/aistudio/work/wolf_data.xlsx', index=False)


In [60]:
import numpy as np

# 目标函数
def objective_function_with_min_display(solution, min_display_weight=2.5, lambda_penalty=300):
    selected_items = merged_data.iloc[solution, :]
    revenue = np.sum(
        selected_items['预测销量'] * 
        selected_items['预测价格'] * 
        selected_items['预测利润率'] * 
        (1 - selected_items['损耗率(%)'] / 100)
    )
    
    # 惩罚项：未满足最小陈列量
    penalty_display = np.sum(lambda_penalty * (min_display_weight - selected_items['预测销量'])**2 * (selected_items['预测销量'] < min_display_weight))

    # 总收益 = 初始收益 - 惩罚项
    final_revenue = revenue - penalty_display
    
    return final_revenue


# 灰狼优化算法
def grey_wolf_optimizer(max_iter=20, n_wolves=50, lower_bound=27, upper_bound=33):
    # 初始化灰狼位置（解）
    wolves = [np.random.choice(merged_data.index, np.random.randint(lower_bound, upper_bound), replace=False) for _ in range(n_wolves)]
    alpha, beta, delta = None, None, None
    
    for iter in range(max_iter):
        # 计算适应度并找到Alpha, Beta, Delta狼
        fitness_values = [objective_function_with_min_display(wolf) for wolf in wolves]
        alpha, beta, delta = sorted(zip(wolves, fitness_values), key=lambda x: x[1], reverse=True)[:3]
        
        # 更新狼群位置
        a = 2 - iter * (2 / max_iter)
        
        for i in range(n_wolves):
            for wolf in [alpha, beta, delta]:
                r1, r2 = np.random.rand(2)
                A = 2 * a * r1 - a
                C = 2 * r2
                
                # 灰狼位置更新逻辑（这里简化为添加或删除一个单品）
                if np.random.rand() < 0.5:
                    if len(wolves[i]) > lower_bound:
                        wolves[i] = np.setdiff1d(wolves[i], np.random.choice(wolves[i]))  # 删除一个单品
                else:
                    if len(wolves[i]) < upper_bound:
                        wolves[i] = np.append(wolves[i], np.random.choice(np.setdiff1d(merged_data.index, wolves[i])))  # 添加一个单品
        
        # 输出当前Alpha狼（最佳解）的适应度
        print(f"Iteration {iter + 1}: Best Revenue = {alpha[1]}")
    
    return alpha

# 执行灰狼优化算法
best_wolf = grey_wolf_optimizer()
best_items = merged_data.iloc[best_wolf[0], :]

print(f"最优解的目标函数值（即最大收益）: {best_wolf[1]}")
print("选定的最优单品：")
print(best_items['单品名称'])

# 打印或保存每个单品的定价和预测销售量
for index, row in best_items.iterrows():
    print(f"单品名称: {row['单品名称']}, 定价: {row['预测价格']}, 预测销售量: {row['预测销量']}")

Iteration 1: Best Revenue = 929.2359067116241
Iteration 2: Best Revenue = 1635.7903901613772
Iteration 3: Best Revenue = 1442.9559176619405
Iteration 4: Best Revenue = 768.9013152006673
Iteration 5: Best Revenue = 860.9088401544986
Iteration 6: Best Revenue = 860.9088401544986
Iteration 7: Best Revenue = 908.4597398251399
Iteration 8: Best Revenue = 988.9070126081767
Iteration 9: Best Revenue = 988.9070126081767
Iteration 10: Best Revenue = 1015.5510100260028
Iteration 11: Best Revenue = 410.7035769851386
Iteration 12: Best Revenue = 412.5831307037679
Iteration 13: Best Revenue = 391.7841359550382
Iteration 14: Best Revenue = 1045.6564041798765
Iteration 15: Best Revenue = 684.2028504475306
Iteration 16: Best Revenue = 814.9900535894953
Iteration 17: Best Revenue = 865.0969766008059
Iteration 18: Best Revenue = 895.8314694135454
Iteration 19: Best Revenue = 1106.426880639901
Iteration 20: Best Revenue = 1084.8475596798658
最优解的目标函数值（即最大收益）: 1084.8475596798658
选定的最优单品：
1            云南油麦菜